In [1]:
from binance import Client,ThreadedWebsocketManager,ThreadedDepthCacheManager
import pandas as pd
from keys import apiKey, secret
import requests


In [17]:
#!pip install binance-futures-connector


Falta:
    1. Promediar las ordenes de compras para armar bloques
    2. calcular porcentajes entre los primeros dos bloques grandes
    3. Traer info de todas las monedas

## Prueba de libro ordenes

In [128]:
# TRAE PRECIO DE FUTUROS
#price = requests.get(f"https://fapi.binance.com/fapi/v1/ticker/price?symbol={simbolo}").json()

In [2]:
simbolo = "BTCUSDT"
limit = 1000

In [3]:
# Funcion que devuelve precio de un simbolo
def getPrice(simbolo):
    tickers = requests.get(f"https://fapi.binance.com/fapi/v1/ticker/price?symbol={simbolo}").json()
    price = float(tickers["price"])
    return price

#funcion que devuelve tabla de bids o asks
def getTablas(depth,taker): 
    df= pd.DataFrame(depth[taker])
    df.columns=["Precio","Volumen"]
    df = df.apply(pd.to_numeric, axis=1)

    return df

# Funcion que devuelve los 5 mayores volumenes para bids y asks
def mayoresVolumenes(tabla):
    mascara_mayores= tabla.sort_values("Volumen",ascending=False).head().index
    mayoresVol =tabla.iloc[mascara_mayores]
    return mayoresVol

#DEPTH
def getLibroOrdenes(simbolo,limit):
    url = f"https://fapi.binance.com/fapi/v1/depth?symbol={simbolo}&limit={limit}"
    depth= requests.get(url).json()
    
    bidsTable = getTablas(depth,"bids")
    asksTable = getTablas(depth,"asks")
    
    mayores_bids = mayoresVolumenes(bidsTable)
    mayores_asks = mayoresVolumenes(asksTable)
    
    return(bidsTable,asksTable,mayores_bids,mayores_asks)

In [4]:
# Funcion que calcula la diferencia de precio para bids y asks
def dif(take,precio):
    if take > precio:
        
        dif = (take - precio) / take * 100
    else:
        dif = (precio - take) / precio * 100
        
    return dif

In [7]:
def calculo(simbolo,limit):
    bidsFuture, asksFuture , mayores_bids,mayores_asks = getLibroOrdenes(simbolo,limit)
    price = getPrice(simbolo)
    dif_asks = round(dif(mayores_asks.iloc[0,0],price),4)
    dif_bids = round(dif(mayores_bids.iloc[0,0],price),4)
    return dif_asks,dif_bids, price, bidsFuture, asksFuture , mayores_bids,mayores_asks

In [14]:

    print(mayores_asks + "\t" +mayores_bids)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None

In [9]:
while 1:
    dif_asks,dif_dibs, price, bidsFuture, asksFuture , mayores_bids,mayores_asks = calculo(simbolo,limit)
    print('\r{:^100}'.format(f" asks ${mayores_asks.iloc[0,0]}\t\t Price: {price}\t\t $ dibs: {mayores_bids.iloc[0,0]}"),end="")
    print('\r{:^100}'.format(f" asks ${mayores_asks.iloc[0,0]}  asks %{dif_asks}\t\t\t\t  dibs: ${mayores_bids.iloc[0,0]}  dibs: %{dif_dibs}"),end="")


                   asks $23241.8  asks %0.5512				  dibs: $23000.0  dibs: %0.4919                   

KeyboardInterrupt: 

In [320]:
(mayores_asks.iloc[0,0] - price) / mayores_asks.iloc[0,0] *100

0.466509433962271

In [313]:
bidsFuture, asksFuture , mayores_bids,mayores_asks = getLibroOrdenes(simbolo,limit)
precio = getPrice(simbolo)

### _Por Hacer_

1) Buscar porcentaje que hay entre punto mayor con el punto siguiente
    a) Encontrar punto de mayor volumen
    b) Recorrer hasta buscar siguiente punto mayor

In [165]:
# Formula para ver porcentaje de diferencia 

(precio-mayores_bids.iloc[1,0])/mayores_bids.iloc[1,0]*100

0.33911791451956175

In [172]:
mayores_asks.sort_values("Precio",ascending=False)

,Precio,Volumen
983,21300.0,201.222
982,21299.9,106.066
936,21294.0,68.439
732,21266.7,63.646
595,21250.0,61.048


In [235]:
def difPrice(taker):
    point_ask = taker.iloc[0,0]
    volume = taker.iloc[0,1]
    price = getPrice(simbolo)
    
    dif = (point_ask - price) / point_ask * 100
    
    return (dif,volume,price,taker)

In [250]:
dif(mayores_asks.iloc[0,0],precio)

0.003325573661460058

In [209]:
# Alertas short
dif,volume,price = difPrice(mayores_asks)
##################
point_ask = mayores_asks.iloc[0,0]
volume = mayores_asks.iloc[0,1]
price2 = getPrice(simbolo)
dif2 = (point_ask - price2) / point_ask * 100
####################################################

print(dif2)
print(dif)
print(f"PRECIO S/F: {price2}")

1.1652582159624447
1.1652582159624447
PRECIO S/F: 21051.8


In [207]:
bids= mayores_bids.Precio.iloc[0]
mayor2= mayores_bids.Precio.iloc[1]
if mayor1 > mayor2:
    a = float(mayor1)
    b = float(mayor2)
    dif = (a-b)/a*100
    print(f"diferencia es de: {dif}%")
else:
    a = float(mayor2)
    b = float(mayor1)
    dif = (a-b)/a*100
    print(f"Dif  es de: {dif}%")
        

Dif  es de: 0.12693240375871132%


In [115]:
#FUTURES

url = f"https://fapi.binance.com/fapi/v1/depth?symbol={simbolo}&limit={limit}"
data = requests.get(url).json()

dfBidsF= pd.DataFrame(data["bids"])
dfasksF= pd.DataFrame(data["asks"])

dfBidsF

_GET /fapi/v1/ticker/price_

Responde
~~~
{
  "symbol": "BTCUSDT",
  "price": "6000.01",
  "time": 1589437530011   // Transaction time
}
~~~


In [137]:
while 1:
    price = requests.get(f"https://fapi.binance.com/fapi/v1/ticker/price?symbol={simbolo}").json()
    print(float(price["price"]))

21143.9
21143.9
21143.9
21143.9
21143.2
21140.3
21143.2
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21140.3
21141.2
21140.3
21141.2
21141.1
21141.1
21141.1
21147.2
21147.2
21147.2
21147.2
21147.2
21147.2
21147.2
21143.9
21143.9
21143.9
21143.9
21143.9
21143.9
21141.3
21141.3
21141.3
21141.3
21141.3
21141.2
21141.2
21141.2
21141.1
21141.1
21141.1
21141.1
21141.1
21141.1
21141.1
21141.1
21141.1
21142.1
21142.1
21142.1
21142.1
21142.1


KeyboardInterrupt: 

In [118]:
# Definicion Variables
simbolo = "BTCUSDT"
limit = 100


## SPOT

In [117]:
client= Client(apiKey,secret)

In [81]:
# Funcion que devuelve precio de un simbolo
def getPrecio(client,simbolo):
    tickers = client.get_all_tickers()
    df = pd.DataFrame(tickers)
    df.set_index("symbol",inplace=True)
    dfSimbolo = df.loc[simbolo]
    return dfSimbolo

In [71]:
#funcion que devuelve tabla de bids o asks
def getTablas(depth,taker): 
    df= pd.DataFrame(depth[taker])
    df.columns=["Precio","Volumen"]
    df = df.apply(pd.to_numeric, axis=1)

    return df

In [72]:
# Funcion que devuelve los 5 mayores volumenes para bids y asks
def mayoresVolumenes(tabla):
    mascara_mayores= tabla.sort_values("Volumen",ascending=False).head().index
    mayoresVol =tabla.iloc[mascara_mayores]
    return mayoresVol

In [82]:
#DEPTH
def getLibroOrdenes(client,simbolo):
    depth= client.get_order_book(symbol=simbolo,limit=1000)
    
    bidsTable = getTablas(depth,"bids")
    asksTable = getTablas(depth,"asks")
    
    mayores_bids = mayoresVolumenes(bidsTable)
    mayores_asks = mayoresVolumenes(asksTable)
    
    return(bidsTable,asksTable,mayores_bids,mayores_asks)

In [87]:
bidsTable,asksTable,mayores_bids,mayores_asks = getLibroOrdenes(client,simbolo)

In [84]:
mayores_asks

,Precio,Volumen
943,21313.86,187.66368
660,21276.85,144.35668
583,21264.56,86.61401
883,21308.63,30.95300
737,21291.31,14.34080


In [76]:
print(getPrecio(simbolo))

price    21187.16000000
Name: BTCUSDT, dtype: object


In [77]:
print(mayores_asks.iloc[0,1])
print(mayores_asks.iloc[1,1])

mayores_asks.iloc[0,1] > mayores_asks.iloc[1,1] 

187.66368
144.35668


True

In [147]:
mayor1= mayores_bids.Price.iloc[2]
mayor2= mayores_bids.Price.iloc[3]
if mayor1 > mayor2:
    a = float(mayor1)
    b = float(mayor2)
    dif = (a-b)/a*100
    print(f"diferencia es de: {dif}%")
else:
    a = float(mayor2)
    b = float(mayor1)
    dif = (a-b)/a*100
    print(f"Dif  es de: {dif}%")
        

Dif  es de: 0.004348405600377233%


In [93]:
for buy in depth["bids"]:
    print(buy)
    if float(buy[1]):
        print(buy[1])

['21038.66000000', '0.03783000']
0.03783000
['21038.64000000', '0.00226000']
0.00226000
['21038.34000000', '0.00094000']
0.00094000
['21038.19000000', '0.00237000']
0.00237000
['21038.18000000', '0.20577000']
0.20577000
['21038.17000000', '0.01129000']
0.01129000
['21038.13000000', '0.00766000']
0.00766000
['21037.44000000', '0.11200000']
0.11200000
['21037.43000000', '0.00403000']
0.00403000
['21037.36000000', '0.00609000']
0.00609000
['21037.29000000', '0.00618000']
0.00618000
['21037.23000000', '0.00607000']
0.00607000
['21037.22000000', '0.00667000']
0.00667000
['21037.21000000', '0.00724000']
0.00724000
['21037.12000000', '0.01256000']
0.01256000
['21037.11000000', '0.00734000']
0.00734000
['21037.06000000', '0.10000000']
0.10000000
['21037.00000000', '0.00600000']
0.00600000
['21036.91000000', '0.00294000']
0.00294000
['21036.82000000', '0.04000000']
0.04000000
['21036.80000000', '0.44630000']
0.44630000
['21036.79000000', '1.78281000']
1.78281000
['21036.70000000', '0.01620000']

In [34]:
df_bid = pd.DataFrame(depth["bids"]) # o asks
df_bid.columns=["Price","Volume"]

In [35]:
mascara_mayores_bids= df_bid.sort_values("Volume",ascending=False).head().index

In [36]:
mayores_bids=df_bid.iloc[mascara_mayores_bids]

In [37]:
mayores_bids

,Price,Volume
95,21153.27000000,3.15792000
16,21162.32000000,24.91526000
88,21153.73000000,2.06306000
47,21158.71000000,1.65971000
9,21162.54000000,1.58315000


In [ ]:
def

## Creo tablas de bids y de asks

In [14]:
df_bids

,Precio,Volumen
0,21163.50000000,0.02059000
1,21163.47000000,0.01358000
2,21163.37000000,0.00053000
3,21163.18000000,0.00800000
4,21163.02000000,0.00100000
...,...,...
95,21153.27000000,3.15792000
96,21153.16000000,0.75629000
97,21153.14000000,0.02600000
98,21152.79000000,0.00500000


In [17]:
print(getPrecio(simbolo))

price    21143.75000000
Name: BTCUSDT, dtype: object


## Cliente Futuros

In [19]:
f_client = CMFutures(apiKey,secret)

In [22]:
print(f_client.account())

{'feeTier': 0, 'canTrade': True, 'canDeposit': True, 'canWithdraw': True, 'updateTime': 0, 'assets': [{'asset': 'AAVE', 'walletBalance': '0.00000000', 'unrealizedProfit': '0.00000000', 'marginBalance': '0.00000000', 'maintMargin': '0.00000000', 'initialMargin': '0.00000000', 'positionInitialMargin': '0.00000000', 'openOrderInitialMargin': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000'}, {'asset': 'BTC', 'walletBalance': '0.00000000', 'unrealizedProfit': '0.00000000', 'marginBalance': '0.00000000', 'maintMargin': '0.00000000', 'initialMargin': '0.00000000', 'positionInitialMargin': '0.00000000', 'openOrderInitialMargin': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000'}, {'asset': 'ADA', 'walletBalance': '0.00000000', 'unrealizedProfit': '0.00000000', 'marginBalance': '0.00000000', 'maintMargin

In [93]:
fbook=pd.DataFrame(fclient.book_ticker())

In [94]:
fbook

,symbol,pair,bidPrice,bidQty,askPrice,askQty,time
0,BTCUSD_PERP,BTCUSD,21220.0,3527,21220.1,9156,1658885834180
1,ETHUSD_PERP,ETHUSD,1440.04,17606,1440.05,535,1658885834181
2,LINKUSD_PERP,LINKUSD,6.283,1089,6.284,9,1658885834180
3,BNBUSD_PERP,BNBUSD,251.879,2324,251.880,185,1658885834180
4,TRXUSD_PERP,TRXUSD,0.06551,678,0.06552,1627,1658885834074
5,DOTUSD_PERP,DOTUSD,6.714,773,6.715,63,1658885834177
6,ADAUSD_PERP,ADAUSD,0.46513,770,0.46514,1569,1658885834174
7,EOSUSD_PERP,EOSUSD,1.103,345,1.104,2771,1658885834152
8,LTCUSD_PERP,LTCUSD,53.64,1257,53.65,27,1658885834174
9,BCHUSD_PERP,BCHUSD,117.54,55,117.55,7,1658885834176
